# Submit GW calculation for molecules

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida.orm import load_code
from aiida.plugins import DataFactory, WorkflowFactory
from widgets.cdxml import CdxmlUpload2GnrWidget
from widgets.computational_resources import (
    ProcessResourcesWidget,
    ResourcesEstimatorWidget,
)

# Custom imports.
from widgets.empa_viewer import EmpaStructureViewer

StructureData = DataFactory("core.structure")
Float = DataFactory("core.float")
Dict = DataFactory("core.dict")
Str = DataFactory("core.str")
Int = DataFactory("core.int")
List = DataFactory("core.list")
Bool = DataFactory("core.bool")

In [ ]:
Cp2kMoleculeOptGwWorkChain = WorkflowFactory("nanotech_empa.cp2k.mol_opt_gw")

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.StructureUploadWidget(title="Import from computer"),
        awb.SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    storable=False,
    node_class="StructureData",
)
display(structure_selector)

# Code.
computational_resources = awb.ComputationalResourcesWidget(
    description="CP2K code:", default_calc_job_plugin="cp2k"
)

In [ ]:
# gw_type
gw_type = ipw.Dropdown(
    options=["gpw_std", "gapw_std", "gapw_hq"],
    value="gpw_std",
    description="Protocol:",
    disabled=False,
)


# Multiplicity
multiplicity = ipw.IntText(value=1, description="Multiplicity", disabled=False)
# UKS
uks = ipw.Checkbox(value=False, description="UKS", disabled=False, indent=False)

# spin guess
spins_up = ipw.Text(
    value="", placeholder="1 2 10..13", description="Spins U:", disabled=False
)

spins_down = ipw.Text(
    value="", placeholder="3 4 14..17", description="Spins D:", disabled=False
)

# geometry mode
geo_opt = ipw.Checkbox(
    value=True, description="Do geo opt", disabled=False, indent=False
)

# description
description = ipw.Text(
    value="",
    placeholder="Calculation description",
    description="Description:",
    disabled=False,
    style={"description_width": "initial"},
)

In [ ]:
def get_builder_gw():
    builder = Cp2kMoleculeOptGwWorkChain.get_builder()

    builder.metadata.description = description.value
    builder.code = load_code(computational_resources.value)
    ase_geom = structure_selector.structure

    # spin guess
    mag_list = [0 for t in ase_geom]
    if uks.value:
        for i in awb.utils.string_range_to_list(spins_up.value)[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(spins_down.value)[0]:
            mag_list[i] = -1

        builder.multiplicity = Int(multiplicity.value)
    builder.structure = structure_selector.structure_node  # StructureData(ase=ase_geom)
    builder.magnetization_per_site = List(list=mag_list)
    builder.protocol = Str(gw_type.value)
    builder.geo_opt = Bool(geo_opt.value)

    # Resources.
    builder.options.geo_opt = {
        "max_wallclock_seconds": resources_scf.walltime_seconds,
        "resources": {
            "num_machines": resources_scf.nodes,
            "num_mpiprocs_per_machine": resources_scf.tasks_per_node,
            "num_cores_per_mpiproc": resources_scf.threads_per_task,
        },
    }
    builder.options.gw = {
        "max_wallclock_seconds": resources_gw.walltime_seconds,
        "resources": {
            "num_machines": resources_gw.nodes,
            "num_mpiprocs_per_machine": resources_gw.tasks_per_node,
            "num_cores_per_mpiproc": resources_gw.threads_per_task,
        },
    }

    return builder

In [ ]:
def after_submission(_=None):
    structure_selector.value = None


btn_submit_gw = awb.SubmitButtonWidget(
    Cp2kMoleculeOptGwWorkChain, inputs_generator=get_builder_gw, path_to_root="../"
)
btn_submit_gw.btn_submit.disabled = True
btn_submit_gw.on_submitted(after_submission)

In [ ]:
output = ipw.Output()


def update_all(_=None):
    btn_submit_gw.btn_submit.disabled = False
    # check system
    only_one_molecule = empa_viewer.details["system_type"] == "Molecule"
    only_one_molecule = (
        only_one_molecule and len(empa_viewer.details["all_molecules"]) == 1
    )
    msg = "GW for this system not implemented"
    if only_one_molecule:
        btn_submit_gw.btn_submit.disabled = False
        msg = ""
    spins_up.value = awb.utils.list_to_string_range(empa_viewer.details["spins_up"])
    spins_down.value = awb.utils.list_to_string_range(empa_viewer.details["spins_down"])
    with output:
        clear_output()
        print(msg)
        if uks.value:
            to_display = [gw_type, geo_opt, uks, spins_up, spins_down, multiplicity]
        else:
            to_display = [gw_type, geo_opt, uks]
        display(ipw.VBox(to_display))


structure_selector.observe(update_all, names="structure")
uks.observe(update_all, names="value")

In [ ]:
# Resources.
resources_scf = ProcessResourcesWidget()
resources_gw = ProcessResourcesWidget()

# Resources estimation.
resources_estimation_scf = ResourcesEstimatorWidget(calculation_type="dft")
resources_estimation_gw = ResourcesEstimatorWidget(calculation_type="gw")

# Link resources widgets.
resources_estimation_scf.link_to_resources_widget(resources_scf)
resources_estimation_gw.link_to_resources_widget(resources_gw)

# Link viewer to resources estimation widgets.
ipw.dlink((empa_viewer, "details"), (resources_estimation_scf, "details"))
ipw.dlink((empa_viewer, "details"), (resources_estimation_gw, "details"))

# Link code selector to resources estimation widgets.
_ = ipw.dlink(
    (computational_resources, "value"), (resources_estimation_scf, "selected_code")
)
_ = ipw.dlink(
    (computational_resources, "value"), (resources_estimation_gw, "selected_code")
)

# Link UKS
_ = ipw.dlink((uks, "value"), (resources_estimation_scf, "uks"))
_ = ipw.dlink((uks, "value"), (resources_estimation_gw, "uks"))

# Estimate all resources
estimate_nodes_button = ipw.Button(
    description="Estimate resources", button_style="warning"
)
estimate_nodes_button.on_click(resources_estimation_scf.estimate_resources)
estimate_nodes_button.on_click(resources_estimation_gw.estimate_resources)

In [ ]:
display(
    output,
    description,
    ipw.HBox(
        [
            ipw.VBox(
                [
                    ipw.HTML("DFT resources"),
                    resources_scf,
                ]
            ),
            ipw.VBox(
                [
                    ipw.HTML("GW resources"),
                    resources_gw,
                ]
            ),
        ]
    ),
    estimate_nodes_button,
    computational_resources,
    btn_submit_gw,
)